In [1]:
# dowloading and unzipping the dataset 
import urllib.request 
import zipfile 
import os 
from pathlib import Path 

url = "https://archive.ics.uci.edu/static/public/228/sms+spam+collection.zip"
zip_path = "sms_spam_collection.zip"
extracted_path = "sms_spam_collection"
data_file_path = Path(extracted_path) / "SMSSpamCollection.tsv"

In [2]:
def download_and_unzip_spam_data(
            url,zip_path,extracted_path,data_file_path
            ):
            if data_file_path.exists():  
                print(f"{data_file_path} already exists. skipping dowload") 
                return 
            with urllib.request.urlopen(url) as respose: 
                with open(zip_path,'wb') as out_file: 
                    out_file.write(respose.read())
            
            with zipfile.ZipFile(zip_path,'r') as zip_ref: 
                zip_ref.extractall(extracted_path) 
            original_file_path = Path(extracted_path) / "SMSSpamCollection" 
            os.rename(original_file_path,data_file_path) 
            print(f"File download and saved as {data_file_path}") 
            

In [3]:
download_and_unzip_spam_data(url, zip_path, extracted_path, data_file_path)

File download and saved as sms_spam_collection/SMSSpamCollection.tsv


In [4]:
import pandas as pd 

df = pd.read_csv(data_file_path,sep='\t',header=None,names=['Label','Text']) 

df.head()

,Label,Text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
# lets look at the class distribution 

print(df['Label'].value_counts())

Label
ham     4825
spam     747
Name: count, dtype: int64


In [6]:
print(df['Label'].value_counts()['ham'] / df['Label'].value_counts()['spam'])
# ham is 6.5 times more than spam messages :( 

6.459170013386881


In [7]:
""" 
For simplicity , and because we prefer a small dataset(which will facilitate faster fine-tuning of the LLM) , 
we choose to undersample the dataset to include 747 instances for each class. 

"""

# creating a balanced dataset 

def create_balanced_dataset(df): 
    num_spam = df[df["Label"] == "spam"].shape[0]  # counts the instances of spam  
    ham_subset = df[df["Label"] == "ham"].sample(n=num_spam, random_state=42)   
    balanced_df = pd.concat([ham_subset, df[df["Label"] == "spam"]]) 

    return balanced_df  

In [8]:
balanced_df = create_balanced_dataset(df) 
print(balanced_df['Label'].value_counts())

Label
ham     747
spam    747
Name: count, dtype: int64


In [9]:
balanced_df['Label'] = balanced_df['Label'].map({'ham':0,'spam':1})  

In [10]:
# splitting the dataset  

def random_split(df,train_frac,validation_frac): 
    df = df.sample(
        frac=1,random_state=123
    ).reset_index(drop=True) 

    train_end = int(len(df) * train_frac) 
    validation_end = train_end + int(len(df) * validation_frac) 

    train_df = df[:train_end] 
    validation_df = df[train_end:validation_end] 
    test_df = df[validation_end:] 
    return train_df, validation_df, test_df

In [11]:
train_df, validation_df, test_df = random_split(
    balanced_df,0.7,0.1) 

In [12]:
train_df.to_csv("train.csv",index=None) 
validation_df.to_csv("validation.csv",index=None) 
test_df.to_csv("test.csv",index=None)

In [13]:
# creating dat loaders 
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")
print(tokenizer.encode("<|endoftext|>", allowed_special={"<|endoftext|>"}))

[50256]


In [14]:
import torch 
from torch.utils.data import Dataset, DataLoader 


In [15]:
class SpamDataset(Dataset): 
    def __init__(self,csv_file,tokenizer,max_length=None,pad_token_id = 50256): 
        self.data = pd.read_csv(csv_file)
        self.encoded_texts = [
            tokenizer.encode(text) for text in self.data['Text']
        ]
        if max_length is None: 
            self.max_length = self._longest_encoded_lenght() 
        else: 
            self.max_length = max_length 
            self.encoded_texts = [
                encoded_text[:self.max_length] 
                for encoded_text in self.encoded_texts
            ] 
        self.encoded_texts = [
            encoded_text + [pad_token_id] * (self.max_length - len(encoded_text))
            for encoded_text in self.encoded_texts 
        ]

    def __getitem__(self,index): 
        encoded = self.encoded_texts[index] 
        label  = self.data.iloc[index]['Label'] 
        return (
            torch.tensor(encoded,dtype=torch.long), 
            torch.tensor(label,dtype=torch.long)
        )
    def __len__(self): 
        return len(self.data)

    def _longest_encoded_lenght(self): 
        max_length = 0 
        for encoded_text in self.encoded_texts: 
            encoded_lenght = len(encoded_text) 
            if encoded_lenght > max_length: 
                max_length = encoded_lenght 
        return max_length 


In [16]:
train_dataset = SpamDataset(
    csv_file = "train.csv", 
    max_length = None , 
    tokenizer = tokenizer
)

In [17]:
print(train_dataset.max_length)

120


In [18]:
val_dataset = SpamDataset(
csv_file="validation.csv",
max_length=train_dataset.max_length,
tokenizer=tokenizer
)
test_dataset = SpamDataset(
csv_file="test.csv",
max_length=train_dataset.max_length,
tokenizer=tokenizer
)

In [19]:
from torch.utils.data import DataLoader 

In [20]:
num_workers = 0 
batch_size = 8 
torch.manual_seed(123) 

train_loader = DataLoader(
    dataset = train_dataset, 
    batch_size = batch_size, 
    shuffle=True, 
    num_workers = num_workers, 
    drop_last=True
)
val_loader = DataLoader(
    dataset = val_dataset, 
    batch_size = batch_size, 
    num_workers = num_workers, 
    drop_last = False
)

test_loader = DataLoader(
    dataset = test_dataset, 
    batch_size = batch_size, 
    num_workers = num_workers, 
    drop_last = False
)

In [21]:
for input_batch, target_batch in train_loader: 
    pass 
print("Input batch dimenstions:",input_batch.shape) 
print("Label batch dimenstions:",target_batch.shape)

Input batch dimenstions: torch.Size([8, 120])
Label batch dimenstions: torch.Size([8])


In [22]:
print(f"{len(train_loader)} training batches")
print(f"{len(val_loader)} validation batches")
print(f"{len(test_loader)} test batches")

130 training batches
19 validation batches
38 test batches


In [23]:
# initialize the model with pretrained weights 
CHOOSE_MODEL = "gpt2-small (124M)"
INPUT_PROMPT = "Every effort moves"

BASE_CONFIG = {
    "vocab_size":50257,  
    "context_length": 1024, 
    "drop_rate":0.0, 
    "qkv_bias":True
}

In [24]:
import torch.nn as nn 


In [25]:
model_configs = {
"gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
"gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
"gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
"gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

In [26]:
BASE_CONFIG.update(model_configs[CHOOSE_MODEL])

In [27]:
# Copyright (c) Sebastian Raschka under Apache License 2.0 (see LICENSE.txt).
# Source for "Build a Large Language Model From Scratch"
#   - https://www.manning.com/books/build-a-large-language-model-from-scratch
# Code: https://github.com/rasbt/LLMs-from-scratch


import os

import requests
import json
import numpy as np
import tensorflow as tf
from tqdm import tqdm


def download_and_load_gpt2(model_size, models_dir):
    # Validate model size
    allowed_sizes = ("124M", "355M", "774M", "1558M")
    if model_size not in allowed_sizes:
        raise ValueError(f"Model size not in {allowed_sizes}")

    # Define paths
    model_dir = os.path.join(models_dir, model_size)
    base_url = "https://openaipublic.blob.core.windows.net/gpt-2/models"
    backup_base_url = "https://f001.backblazeb2.com/file/LLMs-from-scratch/gpt2"
    filenames = [
        "checkpoint", "encoder.json", "hparams.json",
        "model.ckpt.data-00000-of-00001", "model.ckpt.index",
        "model.ckpt.meta", "vocab.bpe"
    ]

    # Download files
    os.makedirs(model_dir, exist_ok=True)
    for filename in filenames:
        file_url = os.path.join(base_url, model_size, filename)
        backup_url = os.path.join(backup_base_url, model_size, filename)
        file_path = os.path.join(model_dir, filename)
        download_file(file_url, file_path, backup_url)

    # Load settings and params
    tf_ckpt_path = tf.train.latest_checkpoint(model_dir)
    settings = json.load(open(os.path.join(model_dir, "hparams.json"), "r", encoding="utf-8"))
    params = load_gpt2_params_from_tf_ckpt(tf_ckpt_path, settings)

    return settings, params


def download_file(url, destination, backup_url=None):
    def _attempt_download(download_url):
        response = requests.get(download_url, stream=True, timeout=60)
        response.raise_for_status()

        file_size = int(response.headers.get("Content-Length", 0))

        # Check if file exists and has same size
        if os.path.exists(destination):
            file_size_local = os.path.getsize(destination)
            if file_size and file_size == file_size_local:
                print(f"File already exists and is up-to-date: {destination}")
                return True

        block_size = 1024  # 1 KB
        desc = os.path.basename(download_url)
        with tqdm(total=file_size, unit="iB", unit_scale=True, desc=desc) as progress_bar:
            with open(destination, "wb") as file:
                for chunk in response.iter_content(chunk_size=block_size):
                    if chunk:
                        file.write(chunk)
                        progress_bar.update(len(chunk))
        return True

    try:
        if _attempt_download(url):
            return
    except requests.exceptions.RequestException:
        if backup_url is not None:
            print(f"Primary URL ({url}) failed. Attempting backup URL: {backup_url}")
            try:
                if _attempt_download(backup_url):
                    return
            except requests.exceptions.RequestException:
                pass

        error_message = (
            f"Failed to download from both primary URL ({url})"
            f"{' and backup URL (' + backup_url + ')' if backup_url else ''}."
            "\nCheck your internet connection or the file availability.\n"
            "For help, visit: https://github.com/rasbt/LLMs-from-scratch/discussions/273"
        )
        print(error_message)
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


# Alternative way using `requests`
"""
def download_file(url, destination):
    # Send a GET request to download the file in streaming mode
    response = requests.get(url, stream=True)

    # Get the total file size from headers, defaulting to 0 if not present
    file_size = int(response.headers.get("content-length", 0))

    # Check if file exists and has the same size
    if os.path.exists(destination):
        file_size_local = os.path.getsize(destination)
        if file_size == file_size_local:
            print(f"File already exists and is up-to-date: {destination}")
            return

    # Define the block size for reading the file
    block_size = 1024  # 1 Kilobyte

    # Initialize the progress bar with total file size
    progress_bar_description = url.split("/")[-1]  # Extract filename from URL
    with tqdm(total=file_size, unit="iB", unit_scale=True, desc=progress_bar_description) as progress_bar:
        # Open the destination file in binary write mode
        with open(destination, "wb") as file:
            # Iterate over the file data in chunks
            for chunk in response.iter_content(block_size):
                progress_bar.update(len(chunk))  # Update progress bar
                file.write(chunk)  # Write the chunk to the file
"""


def load_gpt2_params_from_tf_ckpt(ckpt_path, settings):
    # Initialize parameters dictionary with empty blocks for each layer
    params = {"blocks": [{} for _ in range(settings["n_layer"])]}

    # Iterate over each variable in the checkpoint
    for name, _ in tf.train.list_variables(ckpt_path):
        # Load the variable and remove singleton dimensions
        variable_array = np.squeeze(tf.train.load_variable(ckpt_path, name))

        # Process the variable name to extract relevant parts
        variable_name_parts = name.split("/")[1:]  # Skip the 'model/' prefix

        # Identify the target dictionary for the variable
        target_dict = params
        if variable_name_parts[0].startswith("h"):
            layer_number = int(variable_name_parts[0][1:])
            target_dict = params["blocks"][layer_number]

        # Recursively access or create nested dictionaries
        for key in variable_name_parts[1:-1]:
            target_dict = target_dict.setdefault(key, {})

        # Assign the variable array to the last key
        last_key = variable_name_parts[-1]
        target_dict[last_key] = variable_array

    return params


2025-10-19 11:51:51.498594: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760874711.703181      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760874711.760589      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [28]:

class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(dropout)

        mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
        self.register_buffer("mask", mask)

    def forward(self, x):
        B, T, D_in = x.shape

        Q = self.W_query(x)  # (B, T, D_out)
        K = self.W_key(x)    # (B, T, D_out)
        V = self.W_value(x)  # (B, T, D_out)


        Q = Q.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)  # (B, num_heads, T, head_dim)
        K = K.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        V = V.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)

        # Attention scores
        attn_scores = (Q @ K.transpose(-2, -1)) / (self.head_dim ** 0.5)  # (B, num_heads, T, T)

        mask = self.mask[:T, :T].bool()
        attn_scores = attn_scores.masked_fill(mask[None, None, :, :], float('-inf'))

        # Softmax and dropout
        attn_weights = torch.softmax(attn_scores, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Apply attention
        context = attn_weights @ V  # (B, num_heads, T, head_dim)

        # Merge heads back
        context = context.transpose(1, 2).contiguous().view(B, T, self.d_out)
        context = self.out_proj(context)

        return context


In [29]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
        torch.sqrt(torch.tensor(2.0 / torch.pi)) *
        (x + 0.044715 * torch.pow(x, 3))
        ))

In [30]:
class FeedForward(nn.Module): 
    def __init__(self,cfg:dict): 
        super().__init__() 
        self.layers = nn.Sequential(
            nn.Linear(cfg['emb_dim'],4*cfg['emb_dim']),
            GELU(), 
            nn.Linear(4 * cfg['emb_dim'], cfg['emb_dim'])
        )
    
    def forward(self,x): 
        return self.layers(x) 


In [31]:
class LayerNorm(nn.Module): 
    def __init__(self,emb_dim): 
        super().__init__() 
        self.eps = 1e-5 
        self.scale = nn.Parameter(torch.ones(emb_dim)) 
        self.shift = nn.Parameter(torch.zeros(emb_dim)) 
    
    def forward(self,x): 
        mean = x.mean(dim=-1,keepdim=True) 
        var = x.var(dim=-1,keepdim=True,unbiased=False) 
        norm_x  = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift # we are not forcing them to be gausian , model 
        # can do what it whant here :) 



In [32]:
class TransformerBlock(nn.Module): 
    def __init__(self,cfg:dict):
        super().__init__() 
        self.att = MultiHeadAttention(
            d_in= cfg['emb_dim'],
            d_out = cfg['emb_dim'], 
            context_length=cfg['context_length'], 
            num_heads = cfg['n_heads'], 
            dropout = cfg['drop_rate'], 
            qkv_bias=cfg['qkv_bias'] 
        )
        self.ff = FeedForward(cfg) 
        self.norm1 = LayerNorm(cfg['emb_dim']) 
        self.norm2 = LayerNorm(cfg['emb_dim']) 
        self.drop_shortcut = nn.Dropout(cfg['drop_rate']) 
    
    def forward(self,x): 
        shortcut = x 
        x = self.norm1(x) 
        x = self.att(x) 
        x = self.drop_shortcut(x) 
        x = x + shortcut 

        shortcut = x 
        x = self.norm2(x) 
        x = self.ff(x)  
        x = self.drop_shortcut(x) 
        x = x + shortcut 
        return x 


In [33]:
def assign(left, right):
    if left.shape != right.shape:
        raise ValueError(f"Shape mismatch. Left: {left.shape}, "
        "Right: {right.shape}"
    )
    return torch.nn.Parameter(torch.tensor(right))

In [34]:
class GPTModel(nn.Module): 
    def __init__(self,cfg:dict): 
        super().__init__() 
        self.tok_emb = nn.Embedding(cfg['vocab_size'],cfg['emb_dim']) 
        self.pos_emb = nn.Embedding(cfg['context_length'],cfg['emb_dim']) 
        self.drop_emb = nn.Dropout(cfg['drop_rate']) 

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg['n_layers'])]
        )
        self.final_norm = LayerNorm(cfg['emb_dim']) 

        self.out_head = nn.Linear( 
            cfg['emb_dim'], cfg['vocab_size'],bias=False
        )
    
    def forward(self,in_idx:torch.Tensor): 
        batch_size, sq_len =  in_idx.shape 
        tok_embeds = self.tok_emb(in_idx)  
        pos_embeds = self.pos_emb(
            torch.arange(sq_len,device=in_idx.device)
        )
        x = tok_embeds + pos_embeds 
        x = self.drop_emb(x) 
        x = self.trf_blocks(x) 
        x = self.final_norm(x) 
        logits = self.out_head(x) 
        return logits 

In [35]:
def generate_text_simple(model,idx,max_new_tokens,context_size): 
    for _ in range(max_new_tokens):
        idx_cond = idx[:,-context_size:] 
        with torch.no_grad(): 
            logits = model(idx_cond) 
        logits = logits[:,-1,:]
        probas = torch.softmax(logits,dim=-1) 
        idx_next = torch.argmax(probas,dim=-1,keepdim=True) 
        idx = torch.cat((idx,idx_next),dim=1) 
    return idx 

In [36]:
import numpy as np
def load_weights_into_gpt(gpt, params):
    gpt.pos_emb.weight = assign(gpt.pos_emb.weight, params['wpe'])
    gpt.tok_emb.weight = assign(gpt.tok_emb.weight, params['wte'])
    for b in range(len(params["blocks"])):
        q_w, k_w, v_w = np.split(
            (params["blocks"][b]["attn"]["c_attn"])["w"], 3, axis=-1)
        gpt.trf_blocks[b].att.W_query.weight = assign(
            gpt.trf_blocks[b].att.W_query.weight, q_w.T)
        gpt.trf_blocks[b].att.W_key.weight = assign(
            gpt.trf_blocks[b].att.W_key.weight, k_w.T)
        gpt.trf_blocks[b].att.W_value.weight = assign(
            gpt.trf_blocks[b].att.W_value.weight, v_w.T)
        q_b, k_b, v_b = np.split(
            (params["blocks"][b]["attn"]["c_attn"])["b"], 3, axis=-1)
        gpt.trf_blocks[b].att.W_query.bias = assign(
            gpt.trf_blocks[b].att.W_query.bias, q_b)
        gpt.trf_blocks[b].att.W_key.bias = assign(
            gpt.trf_blocks[b].att.W_key.bias, k_b)
        gpt.trf_blocks[b].att.W_value.bias = assign(
            gpt.trf_blocks[b].att.W_value.bias, v_b)
        gpt.trf_blocks[b].att.out_proj.weight = assign(
            gpt.trf_blocks[b].att.out_proj.weight,
            params["blocks"][b]["attn"]["c_proj"]["w"].T)
        gpt.trf_blocks[b].att.out_proj.bias = assign(
            gpt.trf_blocks[b].att.out_proj.bias,
            params["blocks"][b]["attn"]["c_proj"]["b"])
        gpt.trf_blocks[b].ff.layers[0].weight = assign(
            gpt.trf_blocks[b].ff.layers[0].weight,
            params["blocks"][b]["mlp"]["c_fc"]["w"].T)
        gpt.trf_blocks[b].ff.layers[0].bias = assign(
            gpt.trf_blocks[b].ff.layers[0].bias,
            params["blocks"][b]["mlp"]["c_fc"]["b"])
        gpt.trf_blocks[b].ff.layers[2].weight = assign(
            gpt.trf_blocks[b].ff.layers[2].weight,
            params["blocks"][b]["mlp"]["c_proj"]["w"].T)
        gpt.trf_blocks[b].ff.layers[2].bias = assign(
            gpt.trf_blocks[b].ff.layers[2].bias,
            params["blocks"][b]["mlp"]["c_proj"]["b"])
        gpt.trf_blocks[b].norm1.scale = assign(
            gpt.trf_blocks[b].norm1.scale,
            params["blocks"][b]["ln_1"]["g"])
        gpt.trf_blocks[b].norm1.shift = assign(
            gpt.trf_blocks[b].norm1.shift,
            params["blocks"][b]["ln_1"]["b"])
        gpt.trf_blocks[b].norm2.scale = assign(
            gpt.trf_blocks[b].norm2.scale,
            params["blocks"][b]["ln_2"]["g"])
        gpt.trf_blocks[b].norm2.shift = assign(
            gpt.trf_blocks[b].norm2.shift,
            params["blocks"][b]["ln_2"]["b"])
    gpt.final_norm.scale = assign(gpt.final_norm.scale, params["g"])
    gpt.final_norm.shift = assign(gpt.final_norm.shift, params["b"])
    gpt.out_head.weight = assign(gpt.out_head.weight, params["wte"])

In [37]:
model_size = CHOOSE_MODEL.split(" ")[-1].lstrip("(").rstrip(")")
settings, params = download_and_load_gpt2(
model_size=model_size, models_dir="gpt2"
)
model = GPTModel(BASE_CONFIG)
load_weights_into_gpt(model, params)
model.eval()

checkpoint: 100%|██████████| 77.0/77.0 [00:00<00:00, 157kiB/s]
encoder.json: 100%|██████████| 1.04M/1.04M [00:01<00:00, 692kiB/s] 
hparams.json: 100%|██████████| 90.0/90.0 [00:00<00:00, 166kiB/s]
model.ckpt.data-00000-of-00001: 100%|██████████| 498M/498M [02:06<00:00, 3.94MiB/s]   
model.ckpt.index: 100%|██████████| 5.21k/5.21k [00:00<00:00, 8.41MiB/s]
model.ckpt.meta: 100%|██████████| 471k/471k [00:01<00:00, 467kiB/s]  
vocab.bpe: 100%|██████████| 456k/456k [00:01<00:00, 390kiB/s]  


GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(1024, 768)
  (drop_emb): Dropout(p=0.0, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=True)
        (W_key): Linear(in_features=768, out_features=768, bias=True)
        (W_value): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_shortcut): Dropout(p=0.0, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=7

In [38]:
def text_to_token_ids(text,tokenizer): 
    encoded = tokenizer.encode(text,allowed_special={"<|endoftext|>"}) 
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) # add batch dimension
    return encoded_tensor 

In [39]:
def token_ids_to_text(token_ids:torch.Tensor,tokenizer): 
    flat = token_ids.squeeze(0) # remove batch dimenstion 
    return tokenizer.decode(flat.tolist())

In [40]:
text_1 = "Every effort moves you"
token_ids = generate_text_simple(
model=model,
idx=text_to_token_ids(text_1, tokenizer),
max_new_tokens=15,
context_size=BASE_CONFIG["context_length"]
)
print(token_ids_to_text(token_ids, tokenizer))

Every effort moves you forward.

The first step is to understand the importance of your work


In [41]:
text_2 = (
"Is the following text 'spam'? Answer with 'yes' or 'no':"
" 'You are a winner you have been specially"
" selected to receive $1000 cash or a $2000 award.'"
)
token_ids = generate_text_simple(
model=model,
idx=text_to_token_ids(text_2, tokenizer),
max_new_tokens=23,
context_size=BASE_CONFIG["context_length"]
)
print(token_ids_to_text(token_ids, tokenizer))

Is the following text 'spam'? Answer with 'yes' or 'no': 'You are a winner you have been specially selected to receive $1000 cash or a $2000 award.'

The following text 'spam'? Answer with 'yes' or 'no': 'You are a winner


In [42]:
print(model)

GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(1024, 768)
  (drop_emb): Dropout(p=0.0, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=True)
        (W_key): Linear(in_features=768, out_features=768, bias=True)
        (W_value): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_shortcut): Dropout(p=0.0, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=7

In [43]:
for param in model.parameters():
    param.requires_grad = False

In [44]:
torch.manual_seed(123)
num_classes = 2
model.out_head = torch.nn.Linear(
in_features=BASE_CONFIG["emb_dim"],
out_features=num_classes
)

In [45]:
for param in model.trf_blocks[-1].parameters():
    param.requires_grad = True
for param in model.final_norm.parameters():
    param.requires_grad = True

In [46]:
inputs = tokenizer.encode("Do you have time")
inputs = torch.tensor(inputs).unsqueeze(0)
print("Inputs:", inputs)
print("Inputs dimensions:", inputs.shape)

Inputs: tensor([[5211,  345,  423,  640]])
Inputs dimensions: torch.Size([1, 4])


In [47]:
with torch.no_grad():
    outputs = model(inputs)
    print("Outputs:\n", outputs)
    print("Outputs dimensions:", outputs.shape)

Outputs:
 tensor([[[-1.5854,  0.9904],
         [-3.7235,  7.4548],
         [-2.2661,  6.6049],
         [-3.5983,  3.9902]]])
Outputs dimensions: torch.Size([1, 4, 2])


In [48]:
print("Last output token:", outputs[:, -1, :])

Last output token: tensor([[-3.5983,  3.9902]])


In [49]:
print("Last output token:", outputs[:, -1, :])

Last output token: tensor([[-3.5983,  3.9902]])


In [50]:
probas = torch.softmax(outputs[:, -1, :], dim=-1)
label = torch.argmax(probas)
print("Class label:", label.item())

Class label: 1


In [51]:
logits = outputs[:, -1, :]
label = torch.argmax(logits)
print("Class label:", label.item())

Class label: 1


In [52]:
def calc_accuracy_loader(data_loader, model, device, num_batches=None):
    model.eval() 
    correct_predictions, num_examples=0,0
    if num_batches is None: 
        num_batches = len(data_loader)
    else: 
        num_batches = min(num_batches,len(data_loader))
    for i, (input_batch,target_batch) in enumerate(data_loader): 
        if i < num_batches: 
            input_batch = input_batch.to(device)
            target_batch = target_batch.to(device)
            with torch.no_grad():
                logits = model(input_batch)[:,-1,:] # logits of last ouput token
            predicted_labels  = torch.argmax(logits,dim=-1)
            num_examples += predicted_labels.shape[0] # how many examples in the batch 
            correct_predictions += (
                (predicted_labels == target_batch).sum().item()
            )
        else:
            break
    return correct_predictions / num_examples 

In [53]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
model.to(device) 
torch.manual_seed(123)
train_accuracy = calc_accuracy_loader(
train_loader, model, device, num_batches=10
)
val_accuracy = calc_accuracy_loader(
val_loader, model, device, num_batches=10
)
test_accuracy = calc_accuracy_loader(
test_loader, model, device, num_batches=10
)

In [54]:
def calc_loss_batch(input_batch, target_batch, model, device):
    input_batch = input_batch.to(device)
    target_batch = target_batch.to(device)
    logits = model(input_batch)[:, -1, :]
    loss = torch.nn.functional.cross_entropy(logits, target_batch)
    return loss

In [55]:
def calc_loss_loader(data_loader, model, device, num_batches=None):
    total_loss = 0. 
    if len(data_loader) == 0: 
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        num_batches = min(num_batches,len(data_loader))
    for i, (input_batch,target_batch) in enumerate(data_loader): 
        if i < num_batches:
            loss = calc_loss_batch(
                input_batch,target_batch,model,device
            )
            total_loss += loss.item()
        else:
            break
    return total_loss / num_batches 

In [56]:
with torch.no_grad(): 
    train_loss  = calc_loss_loader(
        train_loader,model,device,num_batches=5
    )
    val_loss = calc_loss_loader(val_loader,model,device,num_batches=5)
    test_loss = calc_loss_loader(test_loader,model,device,num_batches=5) 



In [57]:
print(f"Training loss: {train_loss:.3f}")
print(f"Validation loss: {val_loss:.3f}")
print(f"Test loss: {test_loss:.3f}")

Training loss: 2.450
Validation loss: 2.565
Test loss: 2.325


In [58]:
def train_classifier_simple(
    model,train_loader,val_loader,optimizer,device,
    num_epochs,eval_freq,eval_iter):
    train_losses,val_losses,train_accs,val_accs = [],[],[],[]
    examples_seen,global_step = 0, -1
    for epoch in range(num_epochs):
        model.train()
        for input_batch,target_batch in train_loader:
            optimizer.zero_grad()
            loss = calc_loss_batch(
                input_batch,target_batch,model,device
            )
            loss.backward()
            optimizer.step()
            examples_seen += input_batch.shape[0] # how many examples in the batch
            global_step +=1
            if global_step % eval_freq == 0:
                train_loss, val_loss = evaluate_model(
                    model,train_loader,val_loader,device,eval_iter
                )
                train_losses.append(train_loss)
                val_losses.append(val_losses)
                print(f"Ep {epoch+1} (Step {global_step:06d}): "
                    f"Train loss {train_loss:.3f}, "
                    f"Val loss {val_loss:.3f}"
                    )
        train_accuracy = calc_accuracy_loader(
            val_loader,model,device,num_batches=eval_iter
        )
        val_accuracy = calc_accuracy_loader(
            val_loader,model,device,num_batches=eval_iter
        )
        print(f"Training accuracy: {train_accuracy*100:.2f}% | ", end="")
        print(f"Validation accuracy: {val_accuracy*100:.2f}%")
        train_accs.append(train_accuracy)
        val_accs.append(val_accuracy)
    return train_losses, val_losses, train_accs, val_accs, examples_seen

In [59]:
def evaluate_model(model, train_loader, val_loader, device, eval_iter):
    model.eval()
    with torch.no_grad():
        train_loss = calc_loss_loader(
        train_loader, model, device, num_batches=eval_iter
        )
        val_loss = calc_loss_loader(
        val_loader, model, device, num_batches=eval_iter
        )
    model.train()
    return train_loss, val_loss

In [60]:
import time

In [61]:
start_time = time.time()
torch.manual_seed(123)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5, weight_decay=0.1)
num_epochs = 5
train_losses, val_losses, train_accs, val_accs, examples_seen = \
train_classifier_simple(
model, train_loader, val_loader, optimizer, device,
num_epochs=num_epochs, eval_freq=50,
eval_iter=5
)
end_time = time.time()
execution_time_minutes = (end_time - start_time) / 60
print(f"Training completed in {execution_time_minutes:.2f} minutes.")

Ep 1 (Step 000000): Train loss 2.160, Val loss 2.374
Ep 1 (Step 000050): Train loss 0.654, Val loss 0.631
Ep 1 (Step 000100): Train loss 0.531, Val loss 0.566
Training accuracy: 75.00% | Validation accuracy: 75.00%
Ep 2 (Step 000150): Train loss 0.561, Val loss 0.462
Ep 2 (Step 000200): Train loss 0.537, Val loss 0.390
Ep 2 (Step 000250): Train loss 0.284, Val loss 0.340
Training accuracy: 90.00% | Validation accuracy: 90.00%
Ep 3 (Step 000300): Train loss 0.488, Val loss 0.336
Ep 3 (Step 000350): Train loss 0.291, Val loss 0.305
Training accuracy: 97.50% | Validation accuracy: 97.50%
Ep 4 (Step 000400): Train loss 0.110, Val loss 0.114
Ep 4 (Step 000450): Train loss 0.075, Val loss 0.078
Ep 4 (Step 000500): Train loss 0.012, Val loss 0.051
Training accuracy: 100.00% | Validation accuracy: 100.00%
Ep 5 (Step 000550): Train loss 0.173, Val loss 0.053
Ep 5 (Step 000600): Train loss 0.189, Val loss 0.087
Training accuracy: 97.50% | Validation accuracy: 97.50%
Training completed in 0.64 mi

In [62]:
import matplotlib.pyplot as plt 


In [63]:
def plot_values(
    epochs_seen,examples_seen,train_values,val_values,label="loss"):
    fig, ax1 = plt.subplots(figsize=(5,3))
    ax1.plot(epochs_seen,train_values,label=f"Training {label}")
    ax1.plot(
        epochs_seen,val_values,linestyle="-.",
        label=f"Validation {label}"
    )
    ax1.set_xlabel("Epochs")
    ax1.set_ylabel(label.capitalize())
    ax1.legend()
    ax2 = ax1.twiny()
    ax2.plot(examples_seen,train_values,alpha=0)
    ax2.set_xlabel("Example Seen")
    fig.tight_layout()
    plt.show()

In [ ]:
epochs_tensor = torch.linspace(0, num_epochs, len(train_losses))
examples_seen_tensor = torch.linspace(0, examples_seen, len(train_losses))
plot_values(epochs_tensor, examples_seen_tensor, train_losses, val_losses)

In [ ]:
epochs_tensor = torch.linspace(0, num_epochs, len(train_accs))
examples_seen_tensor = torch.linspace(0, examples_seen, len(train_accs))
plot_values(
epochs_tensor, examples_seen_tensor, train_accs, val_accs,
label="accuracy"
)

In [ ]:
train_accuracy = calc_accuracy_loader(train_loader, model, device)
val_accuracy = calc_accuracy_loader(val_loader, model, device)
test_accuracy = calc_accuracy_loader(test_loader, model, device)
print(f"Training accuracy: {train_accuracy*100:.2f}%")
print(f"Validation accuracy: {val_accuracy*100:.2f}%")
print(f"Test accuracy: {test_accuracy*100:.2f}%")